The **Part 1** of the project is divided into three sections:

1- Feature Extraction (Using SIFT)

2- Outlier Removal (Using RANSAC)

3- Computing the Homographies (Using DLT)


**pip install opencv-python**

**pip install opencv-contrib-python**

**Part 1**

In [127]:
#Imports
from numpy.linalg import eig
import numpy as np
import cv2
import os

In [2]:
#Display the video
capture = cv2.VideoCapture(os.path.abspath('trymefirst_lisbon.mp4'))
while(capture.isOpened()):
    ret, frame = capture.read()
    cv2.imshow('frame',frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):  ##press q if you want the video to stop 
        break
capture.release()
cv2.destroyAllWindows()

The following code **extracts SIFT features** from each frame of the input video

In [85]:

capture = cv2.VideoCapture(os.path.abspath('trymefirst_lisbon.mp4'))

sift_points = [] #nome a definir no config
t = 0 
sift = cv2.SIFT_create(100) #number of sift points
while True:
    t = t + 1
    if t == 5: break 
    success, frame = capture.read()
    if success:
        frame_points = []
        gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
        kp, des = sift.detectAndCompute(gray,None)
        frame_points = ([kp[0].pt[0],kp[0].pt[1]]+des[0].tolist())
        for i in range(1,len(kp)):
             list = ([kp[i].pt[0],kp[i].pt[1]]+des[i].tolist())
             frame_points = np.column_stack((frame_points,list))  
    sift_points.append(frame_points) #append everything into a list 

The following code **matches SIFT features** between the frames

In [113]:
#Brute force method
bf = cv2.BFMatcher(crossCheck=True)
all_matches = []

for s in range(len(sift_points)-1):
    point_matches = []

    des1 = (((sift_points[s])[2:,:])).astype('float32')  # descriptors of the first frame
    des2 = (((sift_points[s+1])[2:,:])).astype('float32')  # descriptors of the second
    des1 = np.reshape(des1,(np.shape(des1)[1],128))
    des2 = np.reshape(des2,(np.shape(des2)[1],128))

    if np.shape(des1)[0] > np.shape(des2)[0]:
             des1 = des1[:-abs(np.shape(des1)[0]-np.shape(des2)[0]),:]  # we are removing the last points so that we have an equal amount of SIFT features between two frames
    if np.shape(des1)[0] < np.shape(des2)[0]:
             des2 = des2[:-abs(np.shape(des1)[0]-np.shape(des2)[0]),:]
    try:
        matches = bf.match(des1,des2)  # an error occurs if two frames have different amounts of SIFT features
    except: 
          print('Error!')        
             
    for i in range(len(matches)):
       
        point_matches.append([matches[i].queryIdx,matches[i].trainIdx])
    all_matches.append(point_matches)

The following code **computes the Homography** between the frames of the video

In [142]:
image1 = sift_points[0]
image2 = sift_points[1]
A=[]
for i in range(len(all_matches[0])):

   x1,y1 = image1[0,all_matches[0][i][0]],image1[1,all_matches[0][i][1]]
   x2,y2 = image1[0,all_matches[0][i][0]],image1[1,all_matches[0][i][1]]
   A.append([-x1, -y1, -1, 0, 0, 0, x2*x1, x2*y1, x2])
   A.append([0, 0, 0, -x1, -y1, -1, y2*x1, y2*y1, y2])

U, S, Vh = np.linalg.svd(A)
eigenvalue,eigenvector=eig(np.matmul(np.transpose(A),A))
H = np.reshape(eigenvector[0],(3,3))

In [143]:
np.linalg.cond(H)

3.4003532494037625